In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from Stitcher_vp2_without_edges_check import stitch_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time
import os

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')


Connecting celiib@10.28.0.34:3306


In [12]:
pinky.NeuriteStitched()

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices),n_pieces number of unconnected mesh pieces outside the largest mesh piece,largest_piece_perc number of faces percentage of largest mesh piece in respect to total mesh,outside_perc number of faces percentage of mesh outside the biggest mesh piece,n_stitched number of mesh pieces stitched back to main mesh,stitched_addon_perc number of faces percentage of pieces that were stitched back in respect to largest mesh piece,n_unstitched number of mesh pieces remaining unstitched back to main mesh,unstitched_perc number of faces percentage of pieces that were not in respect to largest mesh piece
3,648518346341351467,28696,57266,=BLOB=,=BLOB=,0,0.99854,0.99854,0,0.00000,0,0.00000
3,648518346341351503,11185,22272,=BLOB=,=BLOB=,0,0.99955,0.99955,0,0.00000,0,0.00000
3,648518346341351508,56461,112482,=BLOB=,=BLOB=,0,0.96880,0.96880,0,0.00000,0,0.00000
3,648518346341351509,24880,49518,=BLOB=,=BLOB=,0,0.99879,0.99879,0,0.00000,0,0.00000
3,648518346341351514,37747,75320,=BLOB=,=BLOB=,0,0.99928,0.99928,0,0.00000,0,0.00000
3,648518346341351518,20186,40264,=BLOB=,=BLOB=,0,0.99965,0.99965,0,0.00000,0,0.00000
3,648518346341351523,15522,30788,=BLOB=,=BLOB=,0,1.00000,1.00000,0,0.00000,0,0.00000
3,648518346341351525,45886,91544,=BLOB=,=BLOB=,0,0.99891,0.99891,0,0.00000,0,0.00000
3,648518346341351526,17077,34042,=BLOB=,=BLOB=,0,0.99971,0.99971,0,0.00000,0,0.00000
3,648518346341351531,21508,42872,=BLOB=,=BLOB=,0,0.99977,0.99977,0,0.00000,0,0.00000


In [3]:
@schema
class NeuriteStitched(dj.Computed):
    definition=""" 
    -> pinky.Mesh
    ---
    n_vertices           : bigint           # number of vertices in this mesh
    n_triangles          : bigint           # number of triangles in this mesh
    vertices             : longblob         # x,y,z coordinates of vertices
    triangles            : longblob         # triangles (triplets of vertices)
    n_pieces             : int              # number of unconnected mesh pieces outside the largest mesh piece
    largest_piece_perc   : decimal(6,5)     # number of faces percentage of largest mesh piece in respect to total mesh
    outside_perc         : decimal(6,5)     # number of faces percentage of mesh outside the biggest mesh piece
    n_stitched           : int              # number of mesh pieces stitched back to main mesh
    stitched_addon_perc  : decimal(6,5)     # number of faces percentage of pieces that were stitched back in respect to largest mesh piece
    n_unstitched         : int              # number of mesh pieces remaining unstitched back to main mesh        
    unstitched_perc     : decimal(6,5)     # number of faces percentage of pieces that were not in respect to largest mesh piece
    
    
    """
    
    
    key_source = pinky.Mesh() & pinky.Neurite() & pinky.CurrentSegmentation

    def make(self, key):
        
        
        
        global_time = time.time()
        #get the mesh with the error segments filtered away
        start_time = time.time()
        print(str(key['segment_id']) +  ":")
        my_dict = (pinky.Mesh & pinky.Neurite.proj() & pinky.CurrentSegmentation
                           & key).fetch1()
        print(f"Step 1: Retrieving Mesh and removing error segments: {time.time() - start_time}")
        new_key = dict(segmentation=key["segmentation"],
                       segment_id=key["segment_id"])
        
        segment_id = my_dict["segment_id"]
        verts = my_dict["vertices"]
        faces = my_dict["triangles"]

        
        print("NOT Using a load file for meshes")

        [n_vertices,
         n_triangles,
         vertices,
         triangles,
         n_pieces,
         largest_piece_perc,
         outside_perc,
         n_stitched,
         stitched_addon_perc,
         n_unstitched,
         unstitched_perc] = stitch_neuron(segment_id=segment_id,
                                                      vertices=verts,
                                                      faces=faces,
                                                    import_from_off=False,
                                                     load_meshes_flag = False,
                                                 save_meshes_flag = False,
                                                    pymeshfix_flag = False
                                                   )
        
        
        #insert dummy dictionary into correspondence table
        insert_key = dict(key,
                          n_vertices=n_vertices,
                         n_triangles=n_triangles,
                         vertices=vertices,
                         triangles=triangles,
                         n_pieces=n_pieces,
                         largest_piece_perc=largest_piece_perc,
                         outside_perc=outside_perc,
                         n_stitched=n_stitched,
                         stitched_addon_perc=stitched_addon_perc,
                         n_unstitched=n_unstitched,
                         unstitched_perc=unstitched_perc)
                        
        
        self.insert1(insert_key,skip_duplicates=True)
        
    
                         
                                    

In [4]:
start = time.time()
NeuriteStitched.populate(reserve_jobs=True)
print(time.time() - start)

648518346341351467:
Step 1: Retrieving Mesh and removing error segments: 0.02395153045654297
NOT Using a load file for meshes
generating child and main meshes
------Starting the stitcher vp2-------
There were 4 pieces after mesh split
There were 2 pieces found after size threshold
max_index = 0
max_face_len = 57266
Total time for Mesh Cleansing: 0.8522601127624512
Number of outside child meshes =  0
Main mesh and outside child meshes make up 0.9985353095030515% of original mesh
648518346341355189:
Step 1: Retrieving Mesh and removing error segments: 0.05332326889038086
NOT Using a load file for meshes
generating child and main meshes
------Starting the stitcher vp2-------
There were 6 pieces after mesh split
There were 1 pieces found after size threshold
max_index = 0
max_face_len = 77868
Total time for Mesh Cleansing: 0.11785292625427246
Number of outside child meshes =  0
Main mesh and outside child meshes make up 0.9995892169448011% of original mesh
648518346341355190:
Step 1: Retri

KeyboardInterrupt: 

In [7]:
(schema.jobs & "table_name='__neurite_stitched'").delete()
#schema.jobs